In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
# default_exp evaluation

<IPython.core.display.Javascript object>

# Evaluation

Evaluators take `ModelPipeline` objects as input and can run several validation regimes on them.

In [3]:
# export
import numpy as np
import pandas as pd
from typing import Tuple
from tqdm.auto import tqdm

from numerai_blocks.dataset import Dataset, create_dataset
from numerai_blocks.postprocessing import FeatureNeutralizer

<IPython.core.display.Javascript object>

## 0. Base

In [4]:
# export
class BaseEvaluator:
    """
    Evaluation functionality that holds for both
    Numerai Classic and Numerai Signals.
    :param era_col: Column name pointing to eras.
    Most commonly "era" for Classic and "friday_date" for Signals.
    :param fast_mode: Will skip compute intensive metrics
    max_exposure, feature neutral mean and TB200 if set to True.
    """
    def __init__(self, era_col: str = "era", fast_mode = False):
        self.era_col = era_col
        self.fast_mode = fast_mode

    def full_evaluation(self,
                        dataset: Dataset,
                        example_col: str,
                        pred_cols: list = None,
                        target_col: str = "target"
                        ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Perform evaluation for each prediction column in the Dataset
        against give target and example prediction column.
        """
        val_stats = pd.DataFrame()
        dataset.dataf = dataset.dataf.fillna(0.5)
        pred_cols = dataset.prediction_cols if not pred_cols else pred_cols
        for col in tqdm(pred_cols, desc="Evaluation: "):
            col_stats = self.evaluation_one_col(dataset=dataset,pred_col=col,
                                                target_col=target_col,
                                                example_col=example_col)
            val_stats = pd.concat([val_stats, col_stats], axis=0)
        return val_stats

    def evaluation_one_col(self, dataset: Dataset, pred_col: str, target_col: str, example_col: str):
        """
        Perform evaluation for one prediction column
        against given target and example prediction column.
        """
        col_stats = pd.DataFrame()
        # Compute stats
        val_corrs = self.per_era_corrs(dataf=dataset.dataf,
                                        pred_col=pred_col,
                                        target_col=target_col
                                       )
        mean, std, sharpe = self.mean_std_sharpe(era_corrs=val_corrs)
        max_drawdown = self.max_drawdown(era_corrs=val_corrs)
        apy = self.apy(era_corrs=val_corrs)
        example_corr = self.example_correlation(dataset=dataset,
                                                pred_col=pred_col,
                                                example_col=example_col
                                                )
        mmc_mean, mmc_std, mmc_sharpe = self.mmc(dataf=dataset.dataf,
                                                 pred_col=pred_col,
                                                 target_col=target_col,
                                                 example_col=example_col
                                                 )

        col_stats.loc[pred_col, "target"] = target_col
        col_stats.loc[pred_col, "mean"] = mean
        col_stats.loc[pred_col, "std"] = std
        col_stats.loc[pred_col, "sharpe"] = sharpe
        col_stats.loc[pred_col, "max_drawdown"] = max_drawdown
        col_stats.loc[pred_col, "apy"] = apy
        col_stats.loc[pred_col, "mmc_mean"] = mmc_mean
        col_stats.loc[pred_col, "mmc_std"] = mmc_std
        col_stats.loc[pred_col, "mmc_sharpe"] = mmc_sharpe
        col_stats.loc[pred_col, "corr_with_example_preds"] = example_corr

        # Compute intensive stats
        if not self.fast_mode:
            max_feature_exposure = self.max_feature_exposure(dataset=dataset, pred_col=pred_col)
            fn_mean = self.feature_neutral_mean(dataset=dataset, pred_col=pred_col)
            tb200_mean, tb200_std, tb200_sharpe = self.tbx_mean_std_sharpe(dataf=dataset.dataf,
                                                                           pred_col=pred_col,
                                                                           target_col=target_col,
                                                                           tb=200
                                                                           )
            col_stats.loc[pred_col, "max_feature_exposure"] = max_feature_exposure
            col_stats.loc[pred_col, "feature_neutral_mean"] = fn_mean
            col_stats.loc[pred_col, "tb200_mean"] = tb200_mean
            col_stats.loc[pred_col, "tb200_std"] = tb200_std
            col_stats.loc[pred_col, "tb200_sharpe"] = tb200_sharpe
        return col_stats

    def per_era_corrs(self, dataf: pd.DataFrame, pred_col: str,
                      target_col: str) -> pd.Series:
        """ Correlation between prediction and target for each era. """
        return dataf.groupby(dataf[self.era_col])\
            .apply(lambda d: self._normalize_uniform(d[pred_col].fillna(0.5))
                   .corr(d[target_col]))

    def mean_std_sharpe(self, era_corrs: pd.Series) -> Tuple[np.float64, np.float64, np.float64]:
        """
        Average, standard deviation and Sharpe ratio for
        correlations per era.
        """
        mean = era_corrs.mean()
        std = era_corrs.std(ddof=0)
        sharpe = mean / std
        return mean.item(), std.item(), sharpe.item()

    @staticmethod
    def max_drawdown(era_corrs: pd.Series) -> np.float64:
        """ Maximum drawdown per era. """
        # Arbitrarily large window
        rolling_max = (era_corrs + 1).cumprod().rolling(window=9000,
                                                        min_periods=1).max()
        daily_value = (era_corrs + 1).cumprod()
        max_drawdown = -((rolling_max - daily_value) / rolling_max).max()
        return max_drawdown

    @staticmethod
    def apy(era_corrs: pd.Series) -> np.float64:
        """ Annual percentage yield. """
        payout_scores = era_corrs.clip(-0.25, 0.25)
        payout_daily_value = (payout_scores + 1).cumprod()
        apy = (
                      (
                              (payout_daily_value.dropna().iloc[-1])
                              ** (1 / len(payout_scores))
                      )
                      ** 49  # 52 weeks of compounding minus 3 for stake compounding lag
                      - 1
              ) * 100
        return apy

    def example_correlation(self, dataset: Dataset,
                            pred_col: str, example_col: str):
        """ Correlations with example predictions. """
        return self.per_era_corrs(dataf=dataset.dataf,
                                  pred_col=pred_col,
                                  target_col=example_col,
                                  ).mean()

    def max_feature_exposure(self, dataset: Dataset, pred_col: str) -> np.float64:
        """ Maximum exposure over all features. """
        max_per_era = dataset.dataf.groupby(self.era_col).apply(
            lambda d: d[dataset.feature_cols].corrwith(d[pred_col]).abs().max())
        max_feature_exposure = max_per_era.mean()
        return max_feature_exposure

    def feature_neutral_mean(self, dataset: Dataset, pred_col: str) -> np.float64:
        """ Feature neutralized mean performance. """
        fn = FeatureNeutralizer(pred_name=pred_col,
                                era_col=self.era_col,
                                proportion=1.0)
        neutralized_dataset = fn.transform(dataset=dataset)
        return neutralized_dataset.dataf[fn.final_col_name].mean()

    def tbx_mean_std_sharpe(self,
                            dataf: pd.DataFrame,
                            pred_col: str,
                            target_col: str,
                            tb: int = 200
                            ) -> Tuple[np.float64, np.float64, np.float64]:
        """
        Calculate Mean, Standard deviation and Sharpe ratio
        when we focus on the x top and x bottom predictions.
        :param tb: How many of top and bottom predictions to focus on.
        TB200 is the most common situation.
        """
        tb_val_corrs = self._score_by_date(dataf=dataf,
                                           columns=[pred_col],
                                           target=target_col,
                                           tb=tb)
        return self.mean_std_sharpe(era_corrs=tb_val_corrs)

    def mmc(self, dataf: pd.DataFrame,
            pred_col: str,
            target_col: str,
            example_col: str
            ) -> Tuple[np.float64, np.float64, np.float64]:
        """ MMC Mean, standard deviation and Sharpe ratio. """
        mmc_scores = []
        corr_scores = []
        for _, x in dataf.groupby(self.era_col):
            series = self.neutralize_series(self._normalize_uniform(x[pred_col]), (x[example_col]))
            mmc_scores.append(np.cov(series, x[target_col])[0, 1] / (0.29 ** 2))
            corr_scores.append(self._normalize_uniform(x[pred_col]).corr(x[target_col]))

        val_mmc_mean = np.mean(mmc_scores)
        val_mmc_std = np.std(mmc_scores)
        corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
        corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)
        return val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe

    @staticmethod
    def neutralize_series(series, by, proportion=1.0):
        scores = series.values.reshape(-1, 1)
        exposures = by.values.reshape(-1, 1)

        # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
        exposures = np.hstack(
            (exposures,
             np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))

        correction = proportion * (exposures.dot(
            np.linalg.lstsq(exposures, scores, rcond=None)[0]))
        corrected_scores = scores - correction
        neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
        return neutralized

    def _score_by_date(self, dataf: pd.DataFrame, columns: list, target: str, tb: int = None):
        """
        Get era correlation based on given tb (x top and bottom predictions).
        :param tb: How many of top and bottom predictions to focus on.
        TB200 is the most common situation.
        """
        unique_eras = dataf[self.era_col].unique()
        computed = []
        for u in unique_eras:
            df_era = dataf[dataf[self.era_col] == u]
            era_pred = np.float64(df_era[columns].values.T)
            era_target = np.float64(df_era[target].values.T)

            if tb is None:
                ccs = np.corrcoef(era_target, era_pred)[0, 1:]
            else:
                tbidx = np.argsort(era_pred, axis=1)
                tbidx = np.concatenate([tbidx[:, :tb], tbidx[:, -tb:]], axis=1)
                ccs = [np.corrcoef(era_target[idx], pred[idx])[0, 1] for idx, pred in zip(tbidx, era_pred)]
                ccs = np.array(ccs)
            computed.append(ccs)
        return pd.DataFrame(np.array(computed), columns=columns, index=dataf[self.era_col].unique())

    @staticmethod
    def _normalize_uniform(df: pd.DataFrame) -> pd.Series:
        """ Normalize predictions uniformly using ranks. """
        x = (df.rank(method="first") - 0.5) / len(df)
        return pd.Series(x, index=df.index)


<IPython.core.display.Javascript object>

## 1. Numerai Classic

In [5]:
#export
class NumeraiClassicEvaluator(BaseEvaluator):
    def __init__(self, era_col: str = "era", fast_mode = False):
        super().__init__(era_col=era_col, fast_mode=fast_mode)

<IPython.core.display.Javascript object>

## 2. Numerai Signals

In [6]:
# export
class NumeraiSignalsEvaluator(BaseEvaluator):
    def __init__(self, era_col: str = "era", fast_mode = False):
        super().__init__(era_col=era_col, fast_mode=fast_mode)

<IPython.core.display.Javascript object>

In [7]:
# slow
dataset = create_dataset("test_assets/example_signals_preds.parquet")
# dataset = create_dataset("test_assets/example_signals_preds_with_features.csv")
dataset.dataf = dataset.dataf.iloc[:-1]
dataset.dataf.columns = [col if not col.startswith("ft") else f"feature_{' '.join(col.split('_')[1:])}"\
                         for col in dataset.dataf.columns]

evaluator = NumeraiSignalsEvaluator(era_col="friday_date")
val_stats = evaluator.full_evaluation(dataset=dataset,
                                      target_col="target_20d",
                                      pred_cols=["prediction", "prediction_random"],
                                      example_col="prediction_random"
                                      )

⚠ Warning: No 'era' column found in DataFrame. 'era' column is mandatory for certain 
numerai-blocks functionality. ⚠

Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

🤖 Neutralized 'prediction' with proportion '1.0' 🤖

New neutralized column = 'prediction_neutralized_1.0'.

⚠ Warning: No 'era' column found in DataFrame. 'era' column is mandatory for certain 
numerai-blocks functionality. ⚠

...

🤖 Neutralized 'prediction_random' with proportion '1.0' 🤖

New neutralized column = 'prediction_random_neutralized_1.0'.

⚠ Warning: No 'era' column found in DataFrame. 'era' column is mandatory for certain 
numerai-blocks functionality. ⚠

✅ Finished step FeatureNeutralizer. Output shape=(1793953, 7). Time taken for step: 
0:00:02.007923. ✅

<IPython.core.display.Javascript object>

In [8]:
val_stats

,target,mean,std,sharpe,max_drawdown,apy,mmc_mean,mmc_std,mmc_sharpe,corr_with_example_preds,max_feature_exposure,feature_neutral_mean,tb200_mean,tb200_std,tb200_sharpe
prediction,target_20d,0.011410,0.018157,0.628409,-0.099899,73.007442,0.008546,0.013305,0.63460,0.001621,NaN,0.5,0.030639,0.058256,0.525938
prediction_random,target_20d,0.000381,0.014507,0.026268,-0.209139,1.360748,-0.000002,0.000157,0.02614,0.999911,NaN,0.5,0.004668,0.049444,0.094403


<IPython.core.display.Javascript object>

--------------------------------------------------

In [11]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 01_download.ipynb.
Converted 02_dataset.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>